# Random forest

No entiende las dependencias temporales. -> Añadir en la misma fila los datos de los días anteriores con los que se quiere predecir.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Normalization: Min-Max Scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import joblib

## Carga de datos

In [ ]:
inPath = "../../1 Preprocesado"

In [ ]:
# Ficheros de datos
polen = pd.read_csv(f"{inPath}/polen/gramineas.csv")
tiempo = pd.read_csv(f"{inPath}/tiempo/Getafe.csv")

# Unimos los datos
datos = pd.merge(polen, tiempo, on='fecha')

In [ ]:
# Constantes
ANTICIPACION = 1 # Número de días de anticipación con los que se predice
NUM_DIAS = 15    # Número de días en base a los que se predice
MODEL_FILE_NAME = f'random_forest{NUM_DIAS}_{ANTICIPACION}.pkl' #Archivo de guardado del modelo

Adaptación de los datos

In [ ]:
### Prepación de los datos para el modelo de Random Forest
# Ordenamos los datos por fecha
datos = datos.sort_values('fecha')

# Creamos las columnas con los datos de los días anteriores
for i in range(1, NUM_DIAS + 1):
    for columna in ['granos_de_polen', 'prec', 'tmin', 'tmax', 'dir', 'velmedia', 'racha', 'sol']:
        # Supress SettingWithCopyWarning
        datos = datos.copy()
        datos[f'{columna}_{i}'] = datos[columna].shift(i)

# Eliminamos los días que no tienen todos los datos
datos = datos.dropna()

Columna para los meses

In [ ]:
# Añadir columna del mes
datos['mes'] = pd.to_datetime(datos['fecha']).dt.month
datos = datos.drop(columns=['fecha'])

In [ ]:
# Normalizamos los datos (no es necesario para Random Forest)
# scaler = StandardScaler()
# datos = pd.DataFrame(scaler.fit_transform(datos), columns=datos.columns)

In [ ]:
# Separamos los datos en entrenamiento y test
X = datos.drop(columns=['granos_de_polen'])
y = datos['granos_de_polen']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train

Visualización. Es importante destacar que la separación en el conjunto de entrenamiento y el conjunto de test no está mezclada. Estos se debe a que al ser datos con dependencias temporales, no se deben mezclar. Si no estaríamos probando el modelo habiendo entrenado con datos posteriores en el tiempo (sería como si pudiera ver el futuro, pero en la realidad no será así).

In [ ]:
# Visualización de los datos (sepamos los datos de entrenamiento y test)
import matplotlib.pyplot as plt
plt.plot(y_train.index, y_train, label='Train')
plt.plot(y_test.index, y_test, label='Test')
plt.legend()
plt.show()

## Entrenamiento (o carga) del modelo

In [ ]:
model = None

Celda de entrenamiento

In [ ]:
# # Creación del modelo
# model = RandomForestRegressor(n_estimators=100, random_state=42)

# # Entrenamiento del modelo
# model.fit(X_train, y_train)

In [ ]:
# Cargar el modelo (si ya está entrenado y no queremos volver a entrenarlo)
if model is None:
  model = joblib.load(MODEL_FILE_NAME)
  print('Modelo cargado')
model

In [ ]:
# Evaluación del modelo utilizando el conjunto de prueba
y_pred = model.predict(X_test)

### Métricas

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

rmse = np.sqrt(mse)
print('RMSE:', rmse)

mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

r2 = r2_score(y_test, y_pred)
print('R2:', r2)

## Visualización de los resultados

Importancia de las variables

In [ ]:
# Importancia de las características
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(X.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, X.columns[indices[f]], importances[indices[f]]))

In [ ]:
# Importancia de las características más importantes (las 10 primeras)
import seaborn as sns
color_palette = sns.color_palette("Spectral", len([X.columns[i] for i in indices[:10]]))
plt.figure(figsize=(15, 7))
sns.barplot(x=[X.columns[i] for i in indices[:10]], y=importances[indices[:10]], palette=color_palette, hue=[X.columns[i] for i in indices[:10]])
plt.title('Importancia de las características')
plt.show()

Predicción sobre el conjunto de datos de test

In [ ]:
# Gráfico de dispersión de los valores reales frente a los predichos
plt.scatter(y_test, y_pred)
plt.xlabel('Valores reales')
plt.ylabel('Valores predichos')
plt.title('Valores reales vs Valores predichos')
# Línea de tendencia
z = np.polyfit(y_test, y_pred, 1)
p = np.poly1d(z)
plt.plot(y_test, p(y_test), color='darkgreen')
plt.show()

## Guardado del modelo

In [ ]:
# Guardar el modelo
joblib.dump(model, MODEL_FILE_NAME)